In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium.webdriver.common.by import By
import numpy as np
import re
import time

In [2]:
lista=["https://www.transfermarkt.es/detailsuche/spielerdetail/suche/38292887",
          "https://www.transfermarkt.es/detailsuche/spielerdetail/suche/38292896",
          "https://www.transfermarkt.es/detailsuche/spielerdetail/suche/38292904",
        "https://www.transfermarkt.es/detailsuche/spielerdetail/suche/38292921",
          "https://www.transfermarkt.es/detailsuche/spielerdetail/suche/38292935",
"https://www.transfermarkt.es/detailsuche/spielerdetail/suche/38292954"]
# 160+154+140+113+78+72

In [11]:
def extract_births_page(link):
    driver = webdriver.Chrome(executable_path='chromedriver.exe')
    driver.get(link)
    content = driver.page_source
    soup = BeautifulSoup(content)
    job_elements = soup.find_all(lambda tag:  tag.get('class') == ['zentriert'])
    job_elements2 = soup.find_all(lambda tag:  tag.get('class') == ['hauptlink'])
    job_elements3 = [job_elements2[j] for j in range(0,len(job_elements2)) if job_elements2[j].text!='Uruguay']
    positions = [job_elements2[j].findParent().findParent()('tr')[-1] for j in range(0,len(job_elements2)) if job_elements2[j].text!='Uruguay'] 
    lista4=[job_elements2[j].text for j in range(0,len(job_elements2)) if job_elements2[j].text!= 'Uruguay']
    lista2=[job_elements[5*x+1] for x in range(1,int(np.floor(len(job_elements)/5)))]
    players=pd.DataFrame({'names': lista4, 'dob': lista2, 'positon': positions})
    driver.close()
    return players, soup
    
def extract_all(links):
    final=pd.DataFrame()
    link1=links
    p, texto=extract_births_page(link1)
    final = final.append(p)
    pages = texto.find_all(lambda tag:  tag.get('class') == ["tm-pagination__list-item"])
    del texto
    for i in range(2,len(pages)+2):
        link2=link1+'/page/'+str(i)
        p, texto=extract_births_page(link2)
        del texto
        final = final.append(p)
    return final

def extract_list(links2):
    df3=pd.DataFrame()
    for i in links2:
        df2=extract_all(i)
        df3=df3.append(df2)
    return df3

In [12]:
df10=extract_list(lista)

C:\Users\guill\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


The length of df2 is:  162 . The length of df3 is:  162
The length of df2 is:  154 . The length of df3 is:  316
The length of df2 is:  148 . The length of df3 is:  464
The length of df2 is:  126 . The length of df3 is:  590
The length of df2 is:  109 . The length of df3 is:  699
The length of df2 is:  128 . The length of df3 is:  827


In [14]:
df10['month']=df10['dob'].astype(str).str[25:27]
df10['year']=df10['dob'].astype(str).str[30:32]

In [15]:
df10.groupby('month').count()

,names,dob,positon,year
month,,,,
01,103,103,103,103
02,102,102,102,102
03,86,86,86,86
04,76,76,76,76
05,92,92,92,92
06,73,73,73,73
07,65,65,65,65
08,66,66,66,66
09,53,53,53,53


In [17]:
df10.to_csv('data/final_tm_uy.csv')